# Object-Oriented Programming, part 3

This notebook deals with the miscellaneous: we'll talk about handling failure in the "standard" JVM way in Scala, how to save time with method arguments, on-the-spot instantiating of abstract types, and enhancing the power of `this`.

## Exceptions

If you've worked with Java, you know about them. If you haven't - those nasty things that crash your program are called exceptions:

In [1]:
val x = null
x.toString

: 

Exactly. As you can see, calling a method on a null reference is wrong, and the JVM, along with your running program, will crash. The technical term is that the JVM **throws** a runtime exception which nobody **catches**. So there's this pair of actions that the JVM can handle: throwing and catching exceptions.

Exceptions have their own special place in the JVM's heart, but at their essence, **exceptions are instances of some special classes**. Their type hierarchy, at a very high level, looks like this:

* `Throwable`, the mother of all things that will crash your program
    * `Exception`, the mother of all **program-related** crashes, e.g. a file can't be found or you divide by zero
        * `RuntimeException`, a special kind of exception whose catching is not mandatory
            * our beloved `NullPointerException` is an example
    * `Error`, the mother of all **JVM-related** crashes (as opposed to program-related), some examples:
        * `StackOverflowError`
        * `OutOfMemoryError`
        
Keep in mind: **exceptions are not a Scala-specific concept.** Scala works with exceptions because it operates on the Java Virtual Machine. Most Scala exceptions are actually already-defined Java exceptions.

### Throwing exceptions

Formula: `throw` followed by _an instance of a `Throwable` type_. Example:

In [2]:
throw new RuntimeException("this is an exception lol")

: 

This is how you can intentionally crash the program. There are a lot of reasons to do it: 
* computation failures
* resources not found
* formats are incorrect 
* etc.

Question: how does a method like below compile, since it doesn't return the type it's supposed to?

In [3]:
def aFunnyMethod: Int = throw new RuntimeException("no int for you!")

defined function aFunnyMethod

If you remember, throwing an exception is an expression which returns `Nothing`, and this can be used as a substitute for ANY type! 

### Catching exceptions

If you throw an exception (or the JVM throws one), you as a programmer have the freedom to _catch_ it. The syntax goes like in the following example:

In [7]:
def getInt(withExceptions: Boolean): Int = 
    if (withExceptions) throw new RuntimeException("No int for you!")
    else 42

try {
    getInt(false)
} catch {
    case e: NullPointerException => println("got an NPE")
    case e: RuntimeException => println("runtime exception")
    case _ => println("some other exception")
} finally {
    2
}

defined function getInt
res6_1: AnyVal = 42

So the syntax is 

    try {
        // code that might throw an exception
    } catch {
        case e: ExceptionType1 => // code for catching ExceptionType1
        case e: ExceptionType2 => // code for catching ExceptionType2
        case _ => // code for catching anything else
    } finally {
        // an optional block of code that will be evaluated no matter what
    }
    
The `try`-`catch`-`finally` is an expression (much like everything else) and will have
* the return type of the `try` block in the case nothing is thrown
* the return type of the `catch` case that is matched

The `finally` block is only for side-effect operations that need to happen after the `try`-`catch` was evaluated. Examples: closing files, releasing sockets etc.

## Default method and constructor arguments

This one is easy and it goes like this:

Life is hard. We programmers have very hard lives. Sometimes life is so hard that a function might be called 99% of the time with the same parameters and yet we need to pass their values explicitly _every single darn time_.

Luckily Scala saves us from this dreadful fate. Say we have a factorial function that we've worked so hard to get tail-recursive:

In [8]:
def factorial(x: Int, acc): Int = {
  if (x <= 1) acc
  else factorial(x - 1, x * acc)
} 

defined function factorial

In order to compute the factorial of a number, we'll always call it like `factorial(something, 1)`, which is not aesthetic and also annoying. So why don't we

In [9]:
def factorial(x: Int, acc: Int = 1): Int = {
  if (x <= 1) acc
  else factorial(x - 1, x * acc)
}

defined function factorial

give `acc` a default value? In this way we'd only need to call `factorial(something)` and the second argument will be implied as `1`. Of course we can also explicitly call `factorial` with both arguments if we really need to.

Smart question: what if we defined

In [10]:
def aFunnyFunction(x: Int = 0, y: Int): Int = x + y

defined function aFunnyFunction

Then what would `aFunnyFunction(2)` be? Which parameter would have the value `2`?

In [10]:
aFunnyFunction(2)

cmd10.sc:1: not enough arguments for method aFunnyFunction: (x: Int, y: Int)Int.
Unspecified value parameter y.
val res10 = aFunnyFunction(2)
                          ^

: 

So of course the above fails. The compiler doesn't know which parameter has that value, since `x` is now a _leading_ argument. So the smart way to solve this is to _name_ your arguments when you call your function, so the compiler knows what's what:

In [14]:
aFunnyFunction(y = 2)

res13: Int = 2

One last thing: **the whole default and named argument scheme works for constructor arguments as well.**

## `This`, level 2



By now you should know what `this` is - the reference to the current instance on which on which we're calling a method or using a field.

### Some good questions to ask
1. Does `this` make sense for Scala `object`s? <br/>No, because there's never more than one instance of the Scala `object` in question, so `this` doesn't make a difference. However, the Scala language doesn't prevent you from using it.
2. Does `this` make sense for traits or abstract classes, since they can't be instantiated directly?<br/>Nonsense, of course it makes sense! Even though abstract classes and traits can't be instantiated directly, their class descendants will be able to, so `this` will still refer to the current instance.
3. Does `this` absolutely have to be used?<br/> Only if you need to break ambiguity, as in:

In [16]:
class A {
    val x: Int = 0
    def f(x: Int) {
        // in general, these two print statements will show different values
        println(x)
        println(this.x)
    }
}

defined class A

### `def this`

In translation: defining auxiliary constructors, or _overloading constructors_. As we discussed in part 1, the class "header"/"definition" is the class' constructor. We can define more constructors with other signatures by using `def this(...)`:

In [18]:
class Rational(n: Int, d: Int) {

    def this(d: Int) = this(1, d)
 
    private[this] def this() = this(0)
} 


defined class Rational

Rules of playing:
* an auxiliary constructor _must_ call another constructor first
* only the primary constructor may call a super-constructor (by class definition using `extends`)
* constructors may have restricted visibility (say, private constructors)

The `[this]` in square brackets is called a visibility restrictor: `private[something]` means the visibility is restricted to `something`. In this case, it's `this` again - unneccessary, we know, but it's for show.

## Anonymous classes

Finally, now that we talked about constructing things in funny ways, let us hit an example directly:

In [19]:
trait Animal {
    def eat: Unit
}

val funnyAnimal: Animal = new Animal {
    override def eat: Unit = "ahahahahah"
}

defined trait Animal
funnyAnimal: Animal = $sess.cmd18Wrapper$Helper$$anon$1@5b942741

It's exactly what it looks like. We instantiated a trait. Shock.

Only that it's not an actual trait that we instantiated. A slightly more complex operation was unfolding:
* the compiler cut out the part that said `Animal { override def... }` and created a class with a funny secret name like `$sess.cmd18Wrapper$Helper$$anon$1` and made that `extend` Animal
* instantiated the funny-named class
* attached the reference to that instance to `funnyAnimal`

This code is called _instantiating an anonymous class_. This works brilliantly in practice when we don't want to bother creating a special class that we would only use once in our code with little fuss - instantiating `Runnable`s for parallel applications works like a charm. 

All you need to to is
* be a good kid and pass in the constructor arguments to the class you're instantiating
* implement all abstract methods

**Instantiating anonymous classes works well with traits, abstract classes and non-abstract classes alike.**